# Homework 4 - Spark

In this homework, we are practicing Apache Spark.

You are required to turn in this notebook as BDM\_HW4\_Spark\_**NetId**.ipynb. You will be asked to complete each task using Apache Spark. Output can be printed in the notebook.

## Task 1 (5 points)

You are asked to implement Homework 3 using Spark. The description is provided below for your convenience.

You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from NYU Classes as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [142]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'

In [123]:
def oneToone(pid,rows):
    if pid ==0:
        next(rows)
    for row in rows:
        sender,receivers = row
        receiver_list = receivers.split(';')
        for receiver in receiver_list:
            yield(sender,receiver)
def emailToName(x):
    tx,rx = x
    tx = tx.split('@')[0]
    rx = rx.split('@')[0]
    tx =' '.join(map(lambda x:x.capitalize(),tx.split('.')))
    rx =' '.join(map(lambda x:x.capitalize(),rx.split('.')))
    return((tx,rx),1)

In [124]:
dfEmail = spark.read.load(EN_FN,format='csv',header = True,inferSchema=True)

In [133]:
dfEmail = dfEmail.select('From','To')
rddEmail = dfEmail.rdd.mapPartitionsWithIndex(oneToone)\
                 .filter(lambda x: '@enron.com' in x[0] and '.' in x[0].split('@')[0]  and ('@enron.com' in x[1] 
                    and '.' in x[1].split('@')[0]))\
                   .map(emailToName)\
                   .reduceByKey(lambda x,y:1)\
                   .map(lambda x: (tuple(sorted(x[0])),1))\
                   .reduceByKey(lambda x,y:x+y)\
                   .filter(lambda x:x[1]>1)\
                   .sortByKey()\
                   .map(lambda x:('recipropcal',x[0][0]+' : '+x[0][1]))
rddEmail.collect()

[('recipropcal', 'Brenda Whitehead : Elizabeth Sager'),
 ('recipropcal', 'Carol Clair : Debra Perlingiere'),
 ('recipropcal', 'Carol Clair : Mark Taylor'),
 ('recipropcal', 'Carol Clair : Richard Sanders'),
 ('recipropcal', 'Carol Clair : Sara Shackleton'),
 ('recipropcal', 'Carol Clair : Tana Jones'),
 ('recipropcal', 'Debra Perlingiere : Kevin Ruscitti'),
 ('recipropcal', 'Drew Fossum : Susan Scott'),
 ('recipropcal', 'Elizabeth Sager : Janette Elbertson'),
 ('recipropcal', 'Elizabeth Sager : Mark Haedicke'),
 ('recipropcal', 'Elizabeth Sager : Mark Taylor'),
 ('recipropcal', 'Elizabeth Sager : Richard Sanders'),
 ('recipropcal', 'Eric Bass : Susan Scott'),
 ('recipropcal', 'Fletcher Sturm : Greg Whalley'),
 ('recipropcal', 'Fletcher Sturm : Sally Beck'),
 ('recipropcal', 'Gerald Nemec : Susan Scott'),
 ('recipropcal', 'Grant Masson : Vince Kaminski'),
 ('recipropcal', 'Greg Whalley : Richard Sanders'),
 ('recipropcal', 'Janette Elbertson : Mark Taylor'),
 ('recipropcal', 'Janette El

## Task 2 (5 points)

You are asked to implement Task 2 of Lab 5. The description is provided below for your convenience.

We’ll be using two NYC open data sets: the SAT Results and the NYC High School Directory data sets. Both can be downloaded from the links below, or from online class resources.

**Dataset**: *Please note that each school is uniquely identified by an DBN code, which should be found on both data sets.*

**SAT_Results.csv**
Source: https://nycopendata.socrata.com/Education/SAT-Results/f9bf-2cp4  
Description: “The most recent school level results for New York City on the SAT. Results are available at the school level for the graduating seniors of 2012.”

**DOE_High_School_Directory_2014-2015.csv**
Source: https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2  
Description: “Directory of NYC High Schools.”

We would like to know how the Math scores vary across bus lines or subway lines serving the schools. Your task is to compute the average Math scores of all schools along each bus line and subway line. You can find the bus and subway lines serving each school in the High School Dictionary as bus and subway columns.

The expected results are two lists:
1. A list of key/value pairs: with bus line as keys, and the average Math scores as values.
2. A list of key/value pairs: with subway line as keys, and the average Math scores as values.

The top ten lines with highest score are shown below.

In [125]:
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'
SAT_FN = 'SAT_Results.csv'

In [126]:
dfScores = spark.read.load(SAT_FN,format='csv',header = True,inferSchema=True)
dfScores = dfScores.select('DBN',
                          dfScores['Num of SAT Test Takers'].cast('int').alias('ntakers'),
                          dfScores['`SAT Math Avg. Score`'].cast('int').alias('score')
                          ).na.drop()
dfScores = dfScores.select('DBN','ntakers',
                           (dfScores.score*dfScores.ntakers).alias('total'))
dfScores.show()

+------+-------+------+
|   DBN|ntakers| total|
+------+-------+------+
|02M047|     16|  6400|
|21K410|    475|207575|
|30Q301|     98| 43120|
|17K382|     59| 22066|
|18K637|     35| 13335|
|32K403|     50| 18300|
|09X365|     54| 18306|
|11X270|     56| 22064|
|05M367|     33| 12078|
|14K404|     68| 24276|
|30Q575|    135| 66420|
|13K336|      9|  3366|
|04M635|     48| 17712|
|24Q264|     89| 40406|
|17K408|     57| 19494|
|19K618|     60| 22260|
|27Q309|     36| 13644|
|32K552|     67| 24388|
|13K499|     72| 26208|
|07X600|     76| 30400|
+------+-------+------+
only showing top 20 rows



In [127]:
dfSchools = spark.read.load(HSD_FN,format='csv',header = True,inferSchema=True)
dfSchools = dfSchools.na.drop(subset=['boro'])
dfSchools = dfSchools.select('dbn','bus','subway')
dfSchools.show(5)

+------+--------------------+--------------------+
|   dbn|                 bus|              subway|
+------+--------------------+--------------------+
|01M292|B39, M14A, M14D, ...|B, D to Grand St ...|
|01M448|M14A, M14D, M15, ...|F to East Broadwa...|
|01M450|M101, M102, M103,...|6 to Astor Place ...|
|01M509|B39, M103, M14A, ...|B, D to Grand St ...|
|01M539|B39, M14A, M14D, ...|F, J, M, Z to Del...|
+------+--------------------+--------------------+
only showing top 5 rows



In [128]:
dfResults = dfSchools.join(dfScores, dfSchools.dbn==dfScores.DBN,how = 'inner')
dfResults.show()
#rddResult=dfResults.toDF().take(5)

+------+--------------------+--------------------+------+-------+------+
|   dbn|                 bus|              subway|   DBN|ntakers| total|
+------+--------------------+--------------------+------+-------+------+
|01M292|B39, M14A, M14D, ...|B, D to Grand St ...|01M292|     29| 11716|
|01M448|M14A, M14D, M15, ...|F to East Broadwa...|01M448|     91| 38493|
|01M450|M101, M102, M103,...|6 to Astor Place ...|01M450|     70| 28140|
|01M509|B39, M103, M14A, ...|B, D to Grand St ...|01M509|     44| 19052|
|01M539|B39, M14A, M14D, ...|F, J, M, Z to Del...|01M539|    159| 91266|
|01M696|M14A, M14D, M21, ...|                 N/A|01M696|    130| 78520|
|02M047|M101, M102, M14A,...|4, 5, Q to 14th S...|02M047|     16|  6400|
|02M288|M104, M11, M31, M...|     C, E to 50th St|02M288|     62| 24366|
|02M294|B39, M103, M14A, ...|B, D to Grand St ...|02M294|     53| 20352|
|02M296|M104, M11, M31, M...|     C, E to 50th St|02M296|     58| 21750|
|02M298|B39, M103, M14A, ...|6, N, Q, R to Can...|0

In [129]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
dfResults = dfResults.withColumn("Bus", explode(split(dfResults.bus, "[,]")))
dfResults.show()

+------+--------+--------------------+------+-------+-----+
|   dbn|     Bus|              subway|   DBN|ntakers|total|
+------+--------+--------------------+------+-------+-----+
|01M292|     B39|B, D to Grand St ...|01M292|     29|11716|
|01M292|    M14A|B, D to Grand St ...|01M292|     29|11716|
|01M292|    M14D|B, D to Grand St ...|01M292|     29|11716|
|01M292|     M15|B, D to Grand St ...|01M292|     29|11716|
|01M292| M15-SBS|B, D to Grand St ...|01M292|     29|11716|
|01M292|     M21|B, D to Grand St ...|01M292|     29|11716|
|01M292|     M22|B, D to Grand St ...|01M292|     29|11716|
|01M292|      M9|B, D to Grand St ...|01M292|     29|11716|
|01M448|    M14A|F to East Broadwa...|01M448|     91|38493|
|01M448|    M14D|F to East Broadwa...|01M448|     91|38493|
|01M448|     M15|F to East Broadwa...|01M448|     91|38493|
|01M448|     M21|F to East Broadwa...|01M448|     91|38493|
|01M448|     M22|F to East Broadwa...|01M448|     91|38493|
|01M448|      M9|F to East Broadwa...|01

In [130]:
dfResults = dfResults.withColumn("subway", split(dfResults.subway, "[to]")[0])


In [131]:
dfResults = dfResults.withColumn("Subway", explode(split(dfResults.subway, "[,]")))

In [132]:
dfResults.show()

+------+--------+------+------+-------+-----+
|   dbn|     Bus|Subway|   DBN|ntakers|total|
+------+--------+------+------+-------+-----+
|01M292|     B39|     B|01M292|     29|11716|
|01M292|     B39|    D |01M292|     29|11716|
|01M292|    M14A|     B|01M292|     29|11716|
|01M292|    M14A|    D |01M292|     29|11716|
|01M292|    M14D|     B|01M292|     29|11716|
|01M292|    M14D|    D |01M292|     29|11716|
|01M292|     M15|     B|01M292|     29|11716|
|01M292|     M15|    D |01M292|     29|11716|
|01M292| M15-SBS|     B|01M292|     29|11716|
|01M292| M15-SBS|    D |01M292|     29|11716|
|01M292|     M21|     B|01M292|     29|11716|
|01M292|     M21|    D |01M292|     29|11716|
|01M292|     M22|     B|01M292|     29|11716|
|01M292|     M22|    D |01M292|     29|11716|
|01M292|      M9|     B|01M292|     29|11716|
|01M292|      M9|    D |01M292|     29|11716|
|01M448|    M14A|    F |01M448|     91|38493|
|01M448|    M14D|    F |01M448|     91|38493|
|01M448|     M15|    F |01M448|   

In [133]:
from pyspark.sql.functions import trim
dfResults = dfResults.withColumn("Subway", trim(dfResults.Subway))\
         .withColumn("Bus", trim(dfResults.Bus))

In [180]:
dfResultsByBus = dfResults.groupBy('Bus').sum('ntakers','total').na.drop()
dfResultsByBus.show()

+-------+------------+----------+
|    Bus|sum(ntakers)|sum(total)|
+-------+------------+----------+
|   Bx33|        1626|    655606|
|   Q101|        5362|   2463249|
|    Q65|        3175|   1471166|
|B44-SBS|        5642|   2496173|
|     Q7|        1037|    424468|
|   Q110|        1953|    902778|
|    B39|        2067|    970608|
|    Q32|        4653|   2132336|
|   Q100|        2256|   1027875|
|    Q16|         455|    202599|
|   M104|        5333|   2430388|
|    Q50|         263|    103227|
|    Q42|         567|    330360|
|   M103|        4469|   1968277|
|    Q28|        1163|    572883|
|     M4|        5518|   2384419|
|    M66|        3586|   1776627|
|     M7|        8248|   3832017|
|    Bx7|        1290|    512349|
|     B4|        3106|   1443173|
+-------+------------+----------+
only showing top 20 rows



In [181]:
dfResultsByBus = dfResultsByBus.withColumn('avg',dfResultsByBus[2]/dfResultsByBus[1])\
              .select('Bus','avg')

In [182]:
dfResultsBySubway = dfResults.groupBy('Subway').sum('ntakers','total')
dfResultsBySubway = dfResultsBySubway.filter(dfResultsBySubway['Subway']!='N/A')
dfResultsBySubway.show()

+------+------------+----------+
|Subway|sum(ntakers)|sum(total)|
+------+------------+----------+
|     7|       19026|   8686593|
|     3|       38353|  20520406|
|     F|       17044|   7656052|
|     Q|       27031|  12262289|
|     E|       28244|  13916659|
|     B|       27152|  11365231|
|     5|       64539|  30892188|
|   SIR|       10085|   4816062|
|     M|       16086|   7294028|
|     L|        7408|   3188933|
|     6|       18300|   8084209|
|     D|       23857|   9993505|
|     C|       19749|   9594749|
|     J|       24177|  11019724|
|     Z|       20748|   9645154|
|     A|       16806|   8365338|
|     N|       20826|   9356851|
|     S|        5506|   2221505|
|     R|       26780|  12097576|
|     G|        7837|   3138115|
+------+------------+----------+
only showing top 20 rows



In [183]:
dfResultsBySubway = dfResultsBySubway.withColumn('avg',dfResultsBySubway[2]/dfResultsBySubway[1])\
              .select('Subway','avg')

In [184]:
dfResultsBySubway.show()

+------+------------------+
|Subway|               avg|
+------+------------------+
|     7| 456.5643330179754|
|     3| 535.0404401220244|
|     F|449.19338183524997|
|     Q| 453.6380082127927|
|     E|492.72974791106077|
|     B| 418.5780421331762|
|     5|  478.659229303212|
|   SIR|477.54705007436786|
|     M| 453.4395126196693|
|     L| 430.4715172786177|
|     6|441.76005464480875|
|     D|418.89193947269143|
|     C| 485.8346751734265|
|     J| 455.7936882160731|
|     Z|464.87150568729515|
|     A|497.75901463762943|
|     N| 449.2869970229521|
|     S| 403.4698510715583|
|     R| 451.7392083644511|
|     G| 400.4229934924078|
+------+------------------+
only showing top 20 rows



In [185]:
listResultsByBus = dfResultsByBus.select('Bus','avg').rdd.map(lambda x: {x[0]:x[1]}).collect()
listResultsBySubway = dfResultsBySubway.select('Subway','avg').rdd.map(lambda x: {x[0]:x[1]}).collect()

In [186]:
listResultsBySubway

[{'7': 456.5643330179754},
 {'3': 535.0404401220244},
 {'F': 449.19338183524997},
 {'Q': 453.6380082127927},
 {'E': 492.72974791106077},
 {'B': 418.5780421331762},
 {'5': 478.659229303212},
 {'SIR': 477.54705007436786},
 {'M': 453.4395126196693},
 {'L': 430.4715172786177},
 {'6': 441.76005464480875},
 {'D': 418.89193947269143},
 {'C': 485.8346751734265},
 {'J': 455.7936882160731},
 {'Z': 464.87150568729515},
 {'A': 497.75901463762943},
 {'N': 449.2869970229521},
 {'S': 403.4698510715583},
 {'R': 451.7392083644511},
 {'G': 400.4229934924078},
 {'1': 492.7112581895975},
 {'4': 513.9322317188984},
 {'2': 491.6119029275809}]

In [189]:
listResultsByBus[:10]

[{'Bx33': 403.20172201722016},
 {'Q101': 459.3899664304364},
 {'Q65': 463.35937007874014},
 {'B44-SBS': 442.4269762495569},
 {'Q7': 409.32304725168757},
 {'Q110': 462.25192012288784},
 {'B39': 469.5732946298984},
 {'Q32': 458.27122286696755},
 {'Q100': 455.6183510638298},
 {'Q16': 445.2725274725275}]